In [ ]:
%reset

In [ ]:
import os, sys

import pandas as pd
import numpy as np
import swifter

import pacmagic
import pacmagic_icu

from importlib import reload
from data_warehouse_utils.dataloader import DataLoader

os.chdir('/home/adam/files/causal_inference')
os.getcwd()

from causal_inference.experiment_generator.initialize_experiment import initialize_experiment
from causal_inference.experiment_generator.initialize_experiment import add_parameter
from causal_inference.experiment_generator.initialize_experiment import _load_data_patients

In [ ]:
# Reloads packages

reload(sys.modules['causal_inference'])
reload(sys.modules['causal_inference.experiment_generator'])
reload(sys.modules['causal_inference.experiment_generator.initialize_experiment'])


from causal_inference.experiment_generator.initialize_experiment import initialize_experiment

In [ ]:
dl = DataLoader()

df, generator = initialize_experiment(dl = dl,
                           n_of_patients = 10,
                           min_length_of_intubation = 24,
                           length_of_time_window_hours = 4)



In [ ]:
generator_test = generator[0:5]

In [ ]:
BMI = ['body_mass_index']

In [ ]:
LAB_VALUES = ['c_reactive_protein',
              'leukocytes',
              'procalcitonin',
              'hemoglobin',
              'creatinine',
              'ureum',
              'chloride',# is this instead of chlorine?
              'sodium',
              'calcium',
              'phosphate',
              'glucose',
              'activated_partial_thromboplastin_time',
              'prothrombin_time_inr',
              'd_dimer',
              'fibrinogen',
              'alanine_transaminase',
              'aspartate_transaminase'
              'lactate_dehydrogenase',
              'albumin',
              'creatine_kinase',
              'gamma_glutamyl_transferase',
              'alkaline_phosphatase',
              'bilirubin_direct']

In [ ]:
BLOOD_GAS = ['pco2', 'po2', 'bicarbonate', 'lactate' ]


In [ ]:
VITAL_SIGNS = ['heart_rate',
               'arterial_blood_pressure_mean',
               'arterial_blood_pressure_diastolic',
               'arterial_blood_pressure_systolic']

In [ ]:
VENTILATOR_VALUES = ['peep',
                     'fio2',
                     'pressure_above_peep',
                     'tidal_volume',
                     'inspiratory_expiratory_ratio',
                     'respiratory_rate' #this doesn't work and the name is correct
                     'lung_compliance',
                     'driving_pressure',
                     'plateau_pressure',
                     'peak_pressure']

In [ ]:
values = pd.DataFrame([])

for idx, row in enumerate(generator_test):

    patient_id = [generator[idx][0]]
    patient_df = generator[idx][1]

    parameter_value = patient_df[['time_window_start', 'time_window_end']].\
        swifter.\
        apply(lambda row: dl.get_single_timestamp(patients = patient_id,
                                                  parameters = LAB_VALUES,
                                                  columns = ['pacmed_name',
                                                             'numerical_value'],
                                                  from_timestamp = row.time_window_start,
                                                  to_timestamp = row.time_window_end).\
              groupby('pacmed_name').\
              mean(numeric_only = False).
              T,
              axis = 1)

    parameter_value = pd.concat([parameter_value[idx] for idx in range(len(parameter_value))])
    parameter_value.reset_index(inplace = True, drop=True)
    values = pd.concat([values, parameter_value])

In [ ]:
PARAMETER_NAME = ['fio2', 'peep', 'pao2']

values = pd.DataFrame([])

for idx, row in df_test.iterows():

    patient_df = generator[idx][1]

    parameter_value = patient_df[['hash_patient_id', 'time_window_start', 'time_window_end']].\
        swifter.\
        apply(lambda row: dl.get_single_timestamp(patients = [row.hash_patient_id],
                                                  parameters = PARAMETER_NAME,
                                                  columns = ['pacmed_name',
                                                             'numerical_value'],
                                                  from_timestamp = row.time_window_start,
                                                  to_timestamp = row.time_window_end).\
              groupby('pacmed_name').\
              mean(numeric_only = False).
              T,
              axis = 1)

    parameter_value = pd.concat([parameter_value[idx] for idx in range(len(parameter_value))])
    parameter_value.reset_index(inplace = True, drop=True)
    values = pd.concat([values, parameter_value])



In [ ]:
patients['bmi_timestamp'] = patients.apply(lambda row: dl.get_single_timestamp(
    patients = [row.hash_patient_id],
    parameters = ['body_mass_index'],
    columns = ['numerical_value'],
    from_timestamp = row.start_timestamp,
    to_timestamp = row.end_timestamp).first(),
              axis = 1)

In [ ]:
#### Procesing proned patients

df_proning = dl.get_range_measurements(parameters= ['position'],
                                       sub_parameters=['position_body'],
                                       columns=['hash_patient_id',
                                                'start_timestamp',
                                                'end_timestamp',
                                                'effective_value',
                                                'is_correct_unit_yn',
                                                'pacmed_origin_hospital',
                                                'fake_admission_id'
                                                ]
                                       )

df_proning.sort_values(by = ['hash_patient_id', 'start_timestamp'],
                       ascending = True,
                       inplace = True)
df_proning.reset_index(drop=True, inplace=True)

In [ ]:

df = pd.DataFrame(np.ones((2,2)))

In [ ]:
df.head()